**Authors:** Jozef Hanč, Dominik Borovský <br> *[Faculty of Science](https://www.upjs.sk/en/faculty-of-science/?prefferedLang=EN), P. J. Šafárik University in Košice, Slovakia* <br> emails: [jozef.hanc@upjs.sk](mailto:jozef.hanc@upjs.sk)
***

# <font color = brown, size=6> Generating quadruple-precision Hake ratio pdf values </font>

<font size=5> Computational tools: </font>  **<font size=5>SageMath</font>** 

---

## Sage procedures and functions

In [ ]:
# python libraries
import numpy as np
import pandas as pd
from numpy import array as v
from io import StringIO
from time import time

import matplotlib.pyplot as plt
#plt.style.use('seaborn-v0_8-whitegrid')
plt.style.use('seaborn-whitegrid')

load('runtime.py')

In [ ]:
# approximate formulas for precisions expressed in bits and decimal places
bits = lambda d:round((d+1)*ln(10)/ln(2)) 
dps = lambda b:round(b*ln(2)/ln(10) - 1)

# accuracy in Python
def abs_errs(df1, df2):
    N = len(df1)
    errors = [abs(df1[i]-df2[i]) for i in range(N)]
    return errors

In [ ]:
# double precision is 15 decimal places
bits(15)

In [ ]:
# quadrupole precision is 33 decimal places precisions
bits(33), dps(113)

# Exact x values - quadrupole precision

In [ ]:
# writing x values with quadruple precision
def write_xdata(filename, n, dh, hh, dps = 33):
    f = open(filename,"w")
    RRf = RealField(round((dps+1)*ln(10)/ln(2)) )
    data = [RRf(QQ(dh)+(QQ(hh)-QQ(dh))/(n-1)*(i-1)) for i in [1..n]]
    for item in data:
        print(item, file=f)
    f.close()

## interval SixSigma $(-4.5,7.5)$

In [ ]:
# writing x values (quadruple precision) 
xmin, xmax = -9/2,15/2
writing = True
for n in [1..4]:
    tic = time()   # start time
    
    if writing: 
        filename = f'xvalues{10^n}.txt'
        write_xdata('data/'+filename, 10^n,xmin,xmax)
        toc = time()-tic; # end time
        print('10^'+str(n)+' x values: runtime =',toc,'s')

# Exact pdf values - quadrupole precision

## Analytic form $a=3/2, b=1$

In [ ]:
# general definition
a, b = var('a,b')
fk(t) = exp(-(a^2+b^2)/2)/(pi*(1+t^2))
q = (b + a*t)/sqrt(1+t^2)
M(a,b,z) = hypergeometric_M(a,b,z)
# definition of f_T
# formula
assume(a*t+b>0)
fM(t) = fk(t)*(1+q*exp(1/2*q^2)*integral(exp(-1/2*x^2),x, 0, q))
fM.show()

In [ ]:
h(t) = (sqrt(2*pi)*(a*t + b)*erf(1/2*sqrt(2)*(a*t + b)/sqrt(t^2 + 1))*e^(1/2*(a*t + b)^2/(t^2 + 1))/sqrt(t^2 + 1) + 2)/2
fT(t) = fk(t)*h(t)

In [ ]:
# cross-check
(fM(t)-fT(t)).canonicalize_radical()

In [ ]:
av, bv = 3/2, 1

In [ ]:
def fTab(t, a=av, b =bv):
    return fT(t).subs(a=av, b=bv)
fTab(t).show()

In [ ]:
fTab(1).show()
fTab(1).n()

In [ ]:
# fTab quadrupole precision
dps = 33
RRf = RealField(round((dps+1)*ln(10)/ln(2)) )
RRf(fTab(1))

In [ ]:
# generating Marsaglia pdf with quadruple precision
def sage_data(n, dh, hh, dps = 33):
    RRf = RealField(round((dps+1)*ln(10)/ln(2)) )
    xdata = [RRf(QQ(dh)+(QQ(hh)-QQ(dh))/(n-1)*(i-1)) for i in [1..n]]
    ydata = [RRf(fTab(xv)) for xv in xdata]
    return ydata

def write_sage_data(filename, n, dh, hh):
    f = open(filename,"w")
    data = sage_data(n,dh,hh, dps=33)
    for item in data:
        print(item, file=f)
    f.close()

In [ ]:
data = sage_data(100, xmin, xmax)
data[-1]-RRf(fTab(15/2))

In [ ]:
# writing 10^n pdf values in files
writing = True
for n in [1..4]:
    tic = time()   # start time
    
    if writing:
        filename = f'Sage_QPpdf{10^n}.txt'
        write_sage_data('data/'+filename, 10^n,xmin,xmax)
        toc = time()-tic; # end time
        print('10^'+str(n)+' values: runtime =',toc,'s')

In [ ]:
fTfast = fast_float(fTab(t),t)

In [ ]:
rtf = runtime('fTfast(1)')

In [ ]:
rt = runtime('fTab(1).n()')

In [ ]:
rtf.average, rt.average

In [ ]:
rt.average/rtf.average

## Fast SageMath Computation - 1000 points

In [ ]:
methods = {'Analytic Sage fast':fTfast}
pdf = {m:f for m,f in methods.items()}

points = np.linspace(xmin, xmax, 1000)

for method in list(pdf.keys()):
    yh = [pdf[method](u) for u in points]
    plt.plot(points,yh)
    
plt.title(method)
plt.show();

In [ ]:
N = 3
rmin, rmax = -9/2,15/2

In [ ]:
import pandas as pd

In [ ]:
# exponents -3, ... , -15
benchmark = pd.DataFrame(index = range(3, 16), 
            columns = methods.keys())
benchmark.insert(0r, 'err', v([f"{10r^(-n):.0e}" for n in range(3, 16)]))

In [ ]:
benchmark

In [ ]:
# points
x = lambda n: np.linspace(rmin, rmax, n)
# pre-calculated all values for 10^n points
dx = {str(10**(n+1)):np.loadtxt(f'data/xvalues{10**(n+1)}.txt', dtype=np.longdouble) for n in range(N)}
dsageQP = {str(10**(n+1)):np.loadtxt(f'data/Sage_QPpdf{10**(n+1)}.txt', dtype=np.longdouble) for n in range(N)}

In [ ]:
dx['10'], dsageQP['10']

In [ ]:
# start time
rp, Np = 3, 1000
tic = time()
points = dx[str(10**N)].astype(np.float64)
for n in range(3, 16):
    eps = 10**(-n)
    rt = runtime('[fTfast(xh) for xh in points]', r=rp, n=Np, output=False)
    print('N =', 10**N, method, ' eps =', '10^'+str(-n))
    dm = [fTfast(xh) for xh in points]
    dp = dsageQP[str(10**N)]
    ch_abs = abs_errs(dm,dp)
    #ch_rel = rel_errs(dm,dp)
    benchmark.at[n,method] = [rt.average, rt.stdev, min(ch_abs), max(ch_abs) 
                              #min(ch_rel), max(ch_rel)
                             ] #stdev
#end time
toc = time()-tic; 
print('runtime =',toc,'s')
print(25*'*')

In [ ]:
show_allrowscols(benchmark,fullcolwidth=True)

In [ ]:
benchmark.to_csv('data/Analytic-Sage-fast.csv', index=False)